In [7]:
import numpy as np

def tweak1(mat):
    matrix_C = mat.copy()
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            matrix_C[i, j] = mat[i, (j+i) % mat.shape[1]]
    return matrix_C

def tweak2(mat):
    matrix_C = mat.copy()
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            matrix_C[i, j] = mat[(j+i) % mat.shape[0], j]
    return matrix_C

floor = -1.0
ceil = 1.0
A = []
for i in range(4):
    data = (ceil-floor) * np.random.rand(128*128) + floor
    A.append(data.reshape(128, 128))

sigma_A = []
for i in range(4):
    sigma_A.append(tweak1(A[i]))

sigma_A_1 = []
sigma_A_2 = []
for i in range(4):
    sigma_A_1.append(sigma_A[i][:, :64])
    sigma_A_2.append(sigma_A[i][:, 64:])
sigma_A_1_write = np.concatenate(sigma_A_1, axis=1)
sigma_A_2_write = np.concatenate(sigma_A_2, axis=1)
with open(f"sigma_A_1.txt", "w") as f:
    f.write(" ".join(map(str, sigma_A_1_write.flatten())))
with open(f"sigma_A_2.txt", "w") as f:
    f.write(" ".join(map(str, sigma_A_2_write.flatten())))


V = []
for i in range(4):
    data = (ceil-floor) * np.random.rand(128*64) + floor
    V.append(data.reshape(128, 64))
res_V = np.concatenate(V, axis=1)
tau_V = []
for i in range(4):
    tau_V.append(tweak2(V[i]))
tau_V_write = np.concatenate(tau_V, axis=1)
with open(f"tau_V.txt", "w") as f:
    f.write(" ".join(map(str, tau_V_write.flatten())))
with open(f"V.txt", "w") as f:
    f.write(" ".join(map(str, res_V.flatten())))

mul_res = []
for i in range(4):
    mul_res.append(A[i] @ V[i])

res_write = np.concatenate(mul_res, axis=1)
# with open(f"mul_V_res.txt", "w") as f:
#     f.write(" ".join(map(str, res_write.flatten())))


mask_L = [None] * 64
for sed in range(1,64):
    mask_L[sed] = np.zeros(32768)
    for i in range(128):
        for j in range(64):
            for k in range(4):
                if (j < 64-sed):
                    mask_L[sed][i*256 + k*64 + j] = 1.0

mask_R = [None] * 64
for sed in range(1,64):
    mask_R[sed] = np.zeros(32768)
    for i in range(128):
        for j in range(64):
            for k in range(4):
                if (j > 64-sed-1):
                    mask_R[sed][i*256 + k*64 + j] = 1.0

O1 = sigma_A_1_write.copy().reshape(32768)
O2 = sigma_A_2_write.copy().reshape(32768)
rot_V = tau_V_write.copy().reshape(32768)
ct0 = O1 * rot_V
for i in range(1,64):
    rot_V = np.roll(rot_V, -256)
    O1 = np.roll(O1, -1)
    O1_mask = O1 * mask_L[i]
    if (i == 1):
        O2 = np.roll(O2, 64-1)
    else:
        O2 = np.roll(O2, -1)
    O2_mask = O2 * mask_R[i]
    mul1 = O1_mask + O2_mask
    ct0 += mul1 * rot_V


O1 = sigma_A_2_write.copy().reshape(32768)
O2 = sigma_A_1_write.copy().reshape(32768)
rot_V = np.roll(rot_V, -256)
ct0 += O1 * rot_V
for i in range(1,64):
    rot_V = np.roll(rot_V, -256)
    O1 = np.roll(O1, -1)
    O1_mask = O1 * mask_L[i]
    if (i == 1):
        O2 = np.roll(O2, 64-1)
    else:
        O2 = np.roll(O2, -1)
    O2_mask = O2 * mask_R[i]
    mul1 = O1_mask + O2_mask
    ct0 += mul1 * rot_V

res_write = res_write.reshape(32768)
error = 0
for i in range(32768):
    error = max(error, abs(res_write[i]-ct0[i]))
print(error)

with open(f"mul_V_res.txt", "w") as f:
    f.write(" ".join(map(str, ct0.flatten())))

1.5987211554602254e-14
